In [ ]:
import os
from dotenv import load_dotenv
import json
from groq import Groq
from openai import OpenAI
import re

# from langchain_openai import OpenAI




In [17]:
load_dotenv()

True

In [18]:

client = Groq(
    api_key=os.getenv("GROQ_API_KEY"),
)

llm = OpenAI()
# client = OpenAI()


In [19]:
def gpt_llm_call(prompt,model):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": "write a haiku about ai"}
        ]
    )
    return completion.choices[0].message.content


In [20]:
def groq_llm_call(prompt,model):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
    )
    return chat_completion.choices[0].message.content

In [21]:
# print(gpt_llm_call("Who are you?","gpt-3.5-turbo"))

In [22]:
# print(gpt_llm_call("How are you","gpt-3.5-turbo"))

In [34]:
print(groq_llm_call("Who are you?","llama3-8b-8192"))

I am LLaMA, an AI assistant developed by Meta AI that can understand and respond to human input in a conversational manner. I'm not a human, but a computer program designed to simulate conversation, answer questions, and even generate text on a wide range of topics.

I'm trained on a massive dataset of text from the internet and can generate responses to your questions, comments, or topics you'd like to discuss. My abilities include:

1. Answering questions: I can process natural language queries and provide relevant answers.
2. Generating text: I can create coherent and engaging text on a given topic or subject.
3. Translating languages: I can translate text from one language to another, although my proficiency may vary depending on the language pair.
4. Summarizing content: I can summarize long pieces of text into shorter, more digestible versions.
5. Offering suggestions: I can provide suggestions or ideas on a particular topic or scenario.

I'm constantly learning and improving my 

In [35]:
print(groq_llm_call("Who are you?","llama3-70b-8192"))

I am LLaMA, an AI assistant developed by Meta AI that can understand and respond to human input in a conversational manner. I'm a large language model trained on a massive dataset of text from the internet, which allows me to generate human-like responses to a wide range of topics and questions.

I can chat with you about anything from science and history to entertainment and culture. I can also help with tasks like language translation, math calculations, and generating text on a given topic.

Keep in mind that I'm a machine, so I don't have personal experiences, emotions, or opinions like humans do. My responses are generated based on patterns and associations in the data I was trained on, so I may not always understand the nuances of human language or context.

That being said, I'm here to help and provide information to the best of my abilities. So feel free to ask me anything, and I'll do my best to assist you!


In [23]:
import pandas as pd

In [72]:
df=pd.read_csv("solvable_numbers.csv")
df.head()

,Solvables,Solutions
0,1 1 1 8,(1+1+1)*8
1,1 1 1 11,(11+1)*(1+1)
2,1 1 1 12,(1+1)*12*1
3,1 1 1 13,(13-1)*(1+1)
4,1 1 2 6,"(1+1)*6*2,(2+1+1)*6"


In [73]:
input = df["Solvables"][0]
print(input)

1 1 1 8


In [74]:
# 5-shot
standard_prompt =f'''Use numbers and basic arithmetic operations (+ - * /) to obtain 24.
Input: 4 4 6 8
Answer: (4 + 8) * (6 - 4) = 24
Input: 2 9 10 12
Answer: 2 * 12 * (10 - 9) = 24
Input: 4 9 10 13
Answer: (13 - 9) * (10 - 4) = 24
Input: 1 4 8 8
Answer: (8 / 4 + 1) * 8 = 24
Input: 5 5 5 9
Answer: 5 + 5 + 5 + 9 = 24
Input: {input}
'''

# 5-shot
cot_prompt = f'''Use numbers and basic arithmetic operations (+ - * /) to obtain 24. Each step, you are only allowed to choose two of the remaining numbers to obtain a new number.
Input: 4 4 6 8
Steps:
4 + 8 = 12 (left: 4 6 12)
6 - 4 = 2 (left: 2 12)
2 * 12 = 24 (left: 24)
Answer: (6 - 4) * (4 + 8) = 24
Input: 2 9 10 12
Steps:
12 * 2 = 24 (left: 9 10 24)
10 - 9 = 1 (left: 1 24)
24 * 1 = 24 (left: 24)
Answer: (12 * 2) * (10 - 9) = 24
Input: 4 9 10 13
Steps:
13 - 10 = 3 (left: 3 4 9)
9 - 3 = 6 (left: 4 6)
4 * 6 = 24 (left: 24)
Answer: 4 * (9 - (13 - 10)) = 24
Input: 1 4 8 8
Steps:
8 / 4 = 2 (left: 1 2 8)
1 + 2 = 3 (left: 3 8)
3 * 8 = 24 (left: 24)
Answer: (1 + 8 / 4) * 8 = 24
Input: 5 5 5 9
Steps:
5 + 5 = 10 (left: 5 9 10)
10 + 5 = 15 (left: 9 15)
15 + 9 = 24 (left: 24)
Answer: ((5 + 5) + 5) + 9 = 24
Input: {input}
'''

In [75]:
result_standard = groq_llm_call(prompt=standard_prompt,model="llama3-70b-8192")

In [76]:
result_cot = groq_llm_call(prompt=cot_prompt,model="llama3-70b-8192")

In [77]:
verify_prompt_standard = f'''Use numbers and basic arithmetic operations (+ - * /) to obtain 24. Given an input and an answer, give a judgement (sure/impossible) if the answer is correct, i.e. it uses each input exactly once and no other numbers, and reach 24.
Input: 4 4 6 8
Answer: (4 + 8) * (6 - 4) = 24
Judge: 
sure
Input: 2 9 10 12
Answer: 2 * 12 * (10 - 9) = 24
Judge: 
sure
Input: 4 9 10 13
Answer: (13 - 9) * (10 - 4) = 24
Judge: 
sure
Input: 4 4 6 8
Answer: (4 + 8) * (6 - 4) + 1 = 25
Judge: 
impossible
Input: 2 9 10 12
Answer: 2 * (12 - 10) = 24
Judge: 
impossible
Input: 4 9 10 13
Answer: (13 - 4) * (10 - 9) = 24
Judge: 
impossible
Input: {input}
Answer: {result_standard}
Judge:'''


In [78]:

verify_prompt_cot = f'''Use numbers and basic arithmetic operations (+ - * /) to obtain 24. Given an input and an answer, give a judgement (sure/impossible) if the answer is correct, i.e. it uses each input exactly once and no other numbers, and reaches 24.
Input: 4 4 6 8
Answer: (4 + 8) * (6 - 4) = 24
Judge: 
sure
Input: 2 9 10 12
Answer: 2 * 12 * (10 - 9) = 24
Judge: 
sure
Input: 4 9 10 13
Answer: (13 - 9) * (10 - 4) = 24
Judge: 
sure
Input: 4 4 6 8
Answer: (4 + 8) * (6 - 4) + 1 = 25
Judge: 
impossible
Input: 2 9 10 12
Answer: 2 * (12 - 10) = 24
Judge: 
impossible
Input: 4 9 10 13
Answer: (13 - 4) * (10 - 9) = 24
Judge: 
impossible
Input: {input}
Answer: {result_cot}
Judge:'''

In [79]:
groq_llm_call(prompt=verify_prompt_cot,model="llama3-70b-8192")

'Judge: sure'

In [80]:
result_cot

"Let's work through it:\n\nSteps:\n8 - 1 = 7 (left: 1 1 7)\n7 - 1 = 6 (left: 1 6)\n6 * 4 = 24 (left: 24) Wait, no! I made a mistake. There is no 4.\n\nLet me try again:\n\nSteps:\n1 + 1 = 2 (left: 1 2 8)\n2 + 8 = 10 (left: 1 10)\n10 + 1 = 11 (left: 11)\n11 + 1 = 12 (left: 12)\n12 * 2 = 24 (left: 24)\n\nAnswer: (((1 + 1) + 8) + 1) + 1) * 2 = 24"

## Creating Dataset for Finetuning Critique function

In [97]:
data = []

In [98]:
df=pd.read_csv("solvable_numbers.csv")
df.head()

,Solvables,Solutions
0,1 1 1 8,(1+1+1)*8
1,1 1 1 11,(11+1)*(1+1)
2,1 1 1 12,(1+1)*12*1
3,1 1 1 13,(13-1)*(1+1)
4,1 1 2 6,"(1+1)*6*2,(2+1+1)*6"


In [99]:
solvables = df["Solvables"].head(10)


In [100]:
solvables

0     1 1 1 8
1    1 1 1 11
2    1 1 1 12
3    1 1 1 13
4     1 1 2 6
5     2 2 2 3
6     2 2 2 4
7     2 2 2 5
8     2 2 2 7
9     2 2 2 8
Name: Solvables, dtype: object

In [ ]:
for input in solvables:
    propose_prompt = f'''
    From the input use numbers and basic arithmetic operations (+ - * /) to obtain 24. Use every number from the input only once. Give the first operation to head towards 24.

    Example
    Input: 2 8 8 14
    Possible next steps:
    2 + 8 = 10 (left: 8 10 14)
    8 / 2 = 4 (left: 4 8 14)
    14 + 2 = 16 (left: 8 8 16)
    2 * 8 = 16 (left: 8 14 16)
    8 - 2 = 6 (left: 6 8 14)
    14 - 8 = 6 (left: 2 6 8)
    14 /  2 = 7 (left: 7 8 8)
    14 - 2 = 12 (left: 8 8 12)

    Give as many possible next steps such that they eventually lead to 24 in the format shown in the example below for the input given below. Ensure only one operation between two numbers is
    Input: {input}
    Possible next steps:
    '''

    output = groq_llm_call(prompt=propose_prompt,model="llama3-70b-8192")

    # Regular expression pattern
    pattern = r"([\d+\-*/ ]+= [\d]+)\s+\(left: ([\d ]+)\)"

    # Find all matches
    matches = re.findall(pattern, output)

    # Output the extracted results
    for equation, remaining in matches:
        print(f"Equation: {equation}, Remaining: {remaining}")

        verify_math_prompt = f"Verify if the maths is correct, {equation}. Answer ONLY Yes or No"
        verified = groq_llm_call(prompt=verify_math_prompt,model="llama3-70b-8192")

        verified = verified.lower()
        if bool(re.search(r"\bno\b", verified)):
            print("Wrong equation")
            continue




        input = remaining
        propose_prompt_2 = f'''
        From the input use numbers and basic arithmetic operations (+ - * /) to obtain 24. Use every number from the input only once. Give the next operation to head towards 24.

        Example
        Input: 8 10 14
        Possible next steps:
        8 + 10 = 18 (left: 18 14)
        14 - 10 = 4 (left: 4 8)



        Give as many possible next steps such that they eventually lead to 24 in the format shown in the example below for the input given below. Ensure only one operation between two numbers is. If its not possible from there say "Not Possible".
        Input: {input}
        Possible next steps:
        '''
        output_2 = groq_llm_call(prompt=propose_prompt_2,model="llama3-70b-8192")
        print(output_2)
        matches_2 = re.findall(pattern, output_2)
        for equation_2, remaining_2 in matches_2:
            print(f"Last stage:\nEquation: {equation_2}, Remaining: {remaining_2}")
            input_2 = remaining_2
            propose_prompt_3 = f'''
            Try every operation possible on the two input numbers.

            Example
            Input: 8 10
            Possible next steps:
            8 + 10 = 18
            8 - 10 = -2
            10 - 8 = 2
            8 * 10 = 80
            8 / 10 = 0.8
            10 / 8 = 1.25
            
            Input: {input_2}
            Possible next steps:
            '''
            output_3 = groq_llm_call(prompt=propose_prompt_3,model="llama3-70b-8192")
            print(output_3)

            if bool(re.search(r"\b24\b", output_3)):
                entry_1 = {
                    "instruction": "Answer in True of False, whether the remaining numbers can lead to 24 upon using each number exacltly once with basic arithmetic operations (+ - * /)",
                    "input": remaining,
                    "output": "True"
                }
                entry_2 = {
                    "instruction": "Answer in True of False, whether the remaining numbers can lead to 24 upon using each number exacltly once with basic arithmetic operations (+ - * /)",
                    "input": remaining_2,
                    "output": "True"
                }
                data.append(entry_1)
                data.append(entry_2)
            else:
                entry_1 = {
                    "instruction": "Answer in True of False, whether the remaining numbers can lead to 24 upon using each number exacltly once with basic arithmetic operations (+ - * /)",
                    "input": remaining,
                    "output": "False"
                }
                entry_2 = {
                    "instruction": "Answer in True of False, whether the remaining numbers can lead to 24 upon using each number exacltly once with basic arithmetic operations (+ - * /)",
                    "input": remaining_2,
                    "output": "False"
                }
                data.append(entry_1)
                data.append(entry_2)
            




Equation:     1 + 1 = 2, Remaining: 1 2 8
A classic "24 game"!

For the input: 1 2 8

Here are possible next steps:

1. 2 + 1 = 3 (left: 3 8)
2. 8 - 2 = 6 (left: 6 1)

Note that these are just intermediate steps, and we still need to get to 24 from here.

Let me know if you'd like me to explore further or if you'd like to take it from here!
Last stage:
Equation:  2 + 1 = 3, Remaining: 3 8
Here are the possible next steps for the input 3 8:

3 + 8 = 11
3 - 8 = -5
8 - 3 = 5
3 * 8 = 24
3 / 8 = 0.375
8 / 3 = 2.67
Last stage:
Equation:  8 - 2 = 6, Remaining: 6 1
Let's try every operation possible on the input numbers 6 and 1:

Possible next steps:

6 + 1 = 7
6 - 1 = 5
1 - 6 = -5
6 * 1 = 6
6 / 1 = 6
1 / 6 = 0.17
Equation:     1 + 1 + 1 = 3, Remaining: 3 8
A simple one!

Possible next steps:
3 + 8 = 11 (left: 11)

Not possible to reach 24 with just two numbers 3 and 8 using basic arithmetic operations (+ - * /) and using each number only once.
Last stage:
Equation: 3 + 8 = 11, Remaining: 11
I

In [102]:
with open("dataset.json", "w") as json_file:
    json.dump(data, json_file, indent=4)